# Setup the environment

In [1]:
// basic docs for C# interactiv is at: https://github.com/dotnet/interactive/tree/main/docs 
// Use XPlot for visualization, see doc at: https://fslab.org/XPlot/
// For several examples of data analysis methods see: https://xamlbrewer.wordpress.com/category/ml-net/
#r "nuget:XPlot.Plotly"
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget:Microsoft.DotNet.Interactive.ExtensionLab,*-*"

using System;
using System.Threading;

using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.Data;
using Microsoft.Data.Analysis; 
using XPlot.Plotly;

Installed package Microsoft.DotNet.Interactive.ExtensionLab version 1.0.0-beta.20561.1

Installed package XPlot.Plotly version 3.0.1

Installed package Microsoft.Data.Analysis version 0.4.0

Installed package Microsoft.ML version 1.5.2

Loaded Microsoft.DotNet.Interactive.ExtensionLab.DataFrameKernelExtension

...

Loaded Microsoft.DotNet.Interactive.ExtensionLab.NteractKernelExtension

Added the `Explore` extension method, which you can use with `IEnumerable<T>` and `IDataView` to view data using the [nteract Data Explorer](https://github.com/nteract/data-explorer).

Loaded Microsoft.DotNet.Interactive.ExtensionLab.RecordTranscriptExtension

Loaded Microsoft.DotNet.Interactive.ExtensionLab.SqlKernelsExtension


Added `mssql` and `sqlite` to the connection types available using the [`#!connect`](https://github.com/dotnet/interactive/blob/main/docs/connect.md) magic command.

# Load Data

In [1]:
#r "C:\GitHub\traffix.net\samples\IcsMonitor\bin\Debug\netcoreapp3.1\IcsMonitor.dll"
using IcsMonitor;

var ctx = new Interactive();
var table = ctx.ReadToConversationTable(@"d:\Captures\modbus\modbus.pcap",@"c:\temp\", CancellationToken.None);
var keys = table.ConversationKeys.Take(10).ToList();
var processor = new IcsMonitor.Modbus.ModbusBiflowProcessor();
var conv = ctx.GetConversations(table, processor).ToList();


In [1]:
var dataFrame = conv.ToDataFrame();


In [1]:


#!linqify dataFrame

In [1]:
dataFrame

index Label_Class Label_Score Key_ProtocolType Key_SourceIpAddress Key_SourcePort Key_DestinationIpAddress Key_DestinationPort ForwardMetrics_Start ForwardMetrics_End ForwardMetrics_Packets ForwardMetrics_Octets ReverseMetrics_Start ReverseMetrics_End ReverseMetrics_Packets ReverseMetrics_Octets Data_UnitId Data_ReadCoilsRequests Data_ReadCoilsBits Data_ReadDiscreteInputsRequests Data_ReadDiscreteInputsBits .. 0 <null> 0 Tcp 10.0.0.57 2387 10.0.0.3 502 2004-08-26 12:01:18Z 2004-08-26 12:01:18Z 2 120 2004-08-26 12:01:18Z 2004-08-26 12:01:18Z 2 120 0 0 0 0 0 1 <null> 0 Tcp 10.0.0.57 2578 10.0.0.3 502 2004-08-26 12:01:21Z 2004-08-26 12:07:47Z 20 1254 2004-08-26 12:01:21Z 2004-08-26 12:07:47Z 12 800 0 0 0 0 0 2 <null> 0 Tcp 10.0.0.57 2579 10.0.0.8 502 2004-08-26 12:05:53Z 2004-08-26 12:06:16Z 6 374 2004-08-26 12:05:53Z 2004-08-26 12:06:16Z 5 302 0 0 0 0 0 3 <null> 0 Tcp 10.0.0.9 3082 10.0.0.3 502 2004-08-26 12:12:06Z 2004-08-26 12:15:03Z 16 998 2004-08-26 12:12:06Z 2004-08-26 12:15:03Z 9 575 0 2 3 0 0 4 <null> 0 Tcp 10.0.0.57 2585 10.0.0.8 502 2004-08-26 12:22:26Z 2004-08-26 12:23:43Z 8 1384 2004-08-26 12:22:26Z 2004-08-26 12:23:43Z 7 422 0 2 6 0 0 5 <null> 0 Tcp 10.0.0.57 4446 10.0.0.8 502 2004-08-26 12:24:25Z 2004-08-26 12:27:00Z 15 902 2004-08-26 12:24:25Z 2004-08-26 12:27:00Z 16 1018 0 0 0 0 0 6 <null> 0 Tcp 192.168.66.235 2582 166.161.16.230 502 2006-07-21 14:24:39Z 2006-07-21 14:26:14Z 169 10842 2006-07-21 14:24:52Z 2006-07-21 14:26:14Z 181 11298 0 14 20 1 0 7 <null> 0 Tcp 10.1.1.234 51411 10.10.5.85 502 2012-07-20 08:54:59Z 2012-07-20 09:30:00Z 39659 2855442 2012-07-20 08:54:59Z 2012-07-20 09:30:00Z 20100 5447812 0 0 0 0 0

# Space-Time diagram for forward and reverse flows

In [1]:
var fwdSpaceTime = conv.Select(x=>(duration:x.ForwardMetrics.Duration.TotalSeconds, octets: x.ForwardMetrics.Octets));
var revSpaceTime = conv.Select(x=>(duration:x.ReverseMetrics.Duration.TotalSeconds, octets: x.ReverseMetrics.Octets));

In [1]:
var graphFwd = new Graph.Scatter { x = fwdSpaceTime.Select(x=>x.duration), y = fwdSpaceTime.Select(x=>x.octets), mode = "markers" };
var graphRev = new Graph.Scatter { x = revSpaceTime.Select(x=>x.duration), y = revSpaceTime.Select(x=>x.octets), mode = "markers" };
var layout = new Layout.Layout { yaxis = new Graph.Yaxis { type="log" } };
var data = new []{ graphFwd, graphRev };

var plot = Chart.Plot(data);
plot.WithLayout(layout);
plot.WithXTitle("Duration");
plot.WithYTitle("Octets");
plot.WithTitle("Duration-Octets Graph");

plot